In [3]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path

print("=" * 60)
print("SOIL PROPERTIES - WARD-LEVEL EXTRACTION (FIXED)")
print("=" * 60)

# Paths
SOIL_DIR = Path('../data/soil')
WARDS_DIR = Path('../data/wards')

# Load soil raster
print("\n📂 Loading soil data...")
soil_path = SOIL_DIR / 'KMC_Soil_Fixed.tif'

with rasterio.open(soil_path) as src:
    print(f"✓ Loaded soil raster")
    print(f"  Bands: {src.count}")
    print(f"  Data type: {src.dtypes[0]}")

    band_names = ['sand_0-5cm', 'clay_0-5cm', 'silt_0-5cm', 'bulk_density']

# Load wards
wards = gpd.read_file(WARDS_DIR / 'kmc_wards_gee_ready.geojson')
print(f"✓ Loaded {len(wards)} wards")

# Extract soil properties per ward
print(f"\n⚙️  Extracting soil properties per ward...")

ward_soil_features = []

with rasterio.open(soil_path) as src:
    for idx, ward in wards.iterrows():
        if idx % 20 == 0:
            print(f"   Ward {idx+1}/{len(wards)}...")

        ward_id = ward.get('WARD', idx)
        ward_name = ward.get('ward_name', ward.get('WARD_NAME', f'Ward_{ward_id}'))

        try:
            # Mask to ward - DON'T specify nodata for uint16
            ward_geom = [ward.geometry.__geo_interface__]
            out_image, out_transform = mask(src, ward_geom, crop=True, filled=False)

            # Get valid pixels (non-zero, reasonable values)
            sand_data = out_image[0]
            clay_data = out_image[1]
            silt_data = out_image[2]
            bd_data = out_image[3]

            # Filter valid values (0-1000 range for soil fractions)
            valid_mask = (sand_data > 0) & (sand_data < 1000)

            if valid_mask.sum() > 0:
                sand = sand_data[valid_mask]
                clay = clay_data[valid_mask]
                silt = silt_data[valid_mask]
                bulk_density = bd_data[valid_mask]

                # Convert g/kg to %
                features = {
                    'ward_id': str(ward_id),
                    'ward_name': ward_name,
                    'sand_pct': float(np.mean(sand)) / 10,
                    'silt_pct': float(np.mean(silt)) / 10,
                    'clay_pct': float(np.mean(clay)) / 10,
                    'bulk_density_g_cm3': float(np.mean(bulk_density)) / 100,
                    'clay_std': float(np.std(clay)) / 10,

                    # Infiltration index
                    'infiltration_index': (
                        (np.mean(sand) * 0.7) +
                        (np.mean(silt) * 0.3) -
                        (np.mean(clay) * 0.8)
                    ) / 1000,

                    # Soil type
                    'soil_type': 'sandy' if np.mean(sand)/10 > 50 else
                                 'clay' if np.mean(clay)/10 > 40 else
                                 'silty' if np.mean(silt)/10 > 40 else 'loamy'
                }
            else:
                # No valid data
                features = {
                    'ward_id': str(ward_id),
                    'ward_name': ward_name,
                    'sand_pct': 0, 'silt_pct': 0, 'clay_pct': 0,
                    'bulk_density_g_cm3': 0, 'clay_std': 0,
                    'infiltration_index': 0,
                    'soil_type': 'unknown'
                }

            ward_soil_features.append(features)

        except Exception as e:
            # Add zero data for failed wards
            ward_soil_features.append({
                'ward_id': str(ward_id),
                'ward_name': ward_name,
                'sand_pct': 0, 'silt_pct': 0, 'clay_pct': 0,
                'bulk_density_g_cm3': 0, 'clay_std': 0,
                'infiltration_index': 0,
                'soil_type': 'unknown'
            })

# Create DataFrame
soil_df = pd.DataFrame(ward_soil_features)
print(f"\n✓ Soil features calculated for {len(soil_df)} wards")
print(f"  Wards with valid data: {(soil_df['sand_pct'] > 0).sum()}")

# Summary (only valid wards)
valid_soil = soil_df[soil_df['sand_pct'] > 0]

if len(valid_soil) > 0:
    print(f"\n📊 KMC SOIL COMPOSITION (Mean %, valid wards only):")
    print(f"   Sand: {valid_soil['sand_pct'].mean():.1f}%")
    print(f"   Silt: {valid_soil['silt_pct'].mean():.1f}%")
    print(f"   Clay: {valid_soil['clay_pct'].mean():.1f}%")
    print(f"   Bulk density: {valid_soil['bulk_density_g_cm3'].mean():.2f} g/cm³")

    print(f"\n🌊 DRAINAGE QUALITY:")
    drainage_dist = valid_soil['soil_type'].value_counts()
    for quality, count in drainage_dist.items():
        print(f"   {quality}: {count} wards ({count/len(valid_soil)*100:.1f}%)")
else:
    print("\n⚠️  No valid soil data extracted")

# Save anyway
soil_df.to_csv(SOIL_DIR / 'ward_soil_features.csv', index=False)
print(f"\n✓ Saved: {SOIL_DIR / 'ward_soil_features.csv'}")

print("\n✅ SOIL EXTRACTION COMPLETE (partial data)")

SOIL PROPERTIES - WARD-LEVEL EXTRACTION (FIXED)

📂 Loading soil data...
✓ Loaded soil raster
  Bands: 4
  Data type: uint16
✓ Loaded 141 wards

⚙️  Extracting soil properties per ward...
   Ward 1/141...
   Ward 21/141...
   Ward 41/141...
   Ward 61/141...
   Ward 81/141...
   Ward 101/141...
   Ward 121/141...
   Ward 141/141...

✓ Soil features calculated for 141 wards
  Wards with valid data: 141

📊 KMC SOIL COMPOSITION (Mean %, valid wards only):
   Sand: 3.1%
   Silt: 93.3%
   Clay: 3.7%
   Bulk density: 1.30 g/cm³

🌊 DRAINAGE QUALITY:
   silty: 141 wards (100.0%)

✓ Saved: ../data/soil/ward_soil_features.csv

✅ SOIL EXTRACTION COMPLETE (partial data)
